In [1]:
import numpy as np
import pennylane as qml
from pennylane import X, Y, Z, I
from kak_tools import identify_algebra, split_pauli_algebra

In [2]:
def run(gens, verbose=True):
    gens = [list(op.pauli_rep)[0] for op in gens] # Map generators from Operator to PauliWord
    dla = qml.lie_closure(gens, pauli=True)
    dla = [list(op)[0] for op in dla] # Map dla from PauliSentence to PauliWord
    np.random.shuffle(dla)
    sub_dlas = split_pauli_algebra(dla, verbose=verbose)
    return identify_algebra(sub_dlas, verbose=verbose)

In [3]:
# Test with so(2n)
n = 10
gens = [X(w) @ X(w+1) for w in range(n-1)] + [Y(w) @ Y(w+1) for w in range(n-1)] + [Z(w) for w in range(n)]
identifiers = run(gens)

Found 190 components with dimensions 190.
Dimension of component: 190.
It can be one of the following:
so(20)


In [9]:
# Test with random generators
np.random.seed(6135)
num_wires = 4
k = 6
words = np.random.choice([I, X, Y, Z], replace=True, size=(k, num_wires))
gens = [qml.prod(*(P(w) for w, P in enumerate(word))) for word in words]
identifiers = run(gens)

Found 36 components with dimensions 36.
Dimension of component: 36.
It can be one of the following:
so(9)
sp(4)
